1.2 default setting

In [2]:
import os
import subprocess
import torch
from transformers import GPT2Tokenizer
import sys
nanoGPT_dir = "C:/Users/Zheng/OneDrive - UW-Madison/PhD project/Coursework/CS839/hw/nanoGPT-master"
sys.path.append(nanoGPT_dir)
os.chdir(nanoGPT_dir)
from model import GPTConfig, GPT

In [2]:
# Prepare the path for the script you want to run
prepare_script = os.path.join("data", "shakespeare_char", "prepare.py")

# Run the script using os.system
os.system(f'python "{prepare_script}"')

# Run scripts
train_script = os.path.join("train.py")
config_path = os.path.join("config", "train_shakespeare_char.py")
command = f'python "{train_script}" "{config_path}" --device=cpu --compile=False --eval_iters=20 --log_interval=1 --block_size=64 --batch_size=12 --n_layer=4 --n_head=4 --n_embd=128 --max_iters=2000 --lr_decay_iters=2000 --dropout=0.0'
result = subprocess.run(command, shell=True, text=True, capture_output=True)
print(result.stdout)

Overriding config with config\train_shakespeare_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-shakespeare-char'
eval_interval = 250 # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False # override via command line if you like
wandb_project = 'shakespeare-char'
wandb_run_name = 'mini-gpt'

dataset = 'shakespeare_char'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of 

In [3]:
# Generate text by sample.py
sample_script = os.path.join("sample.py")
out_dir = "out-shakespeare-char"
command = f'python "{sample_script}" --out_dir="{out_dir}" --device=cpu'
result = subprocess.run(command, shell=True, text=True, capture_output=True)
print(result.stdout)

Overriding: out_dir = out-shakespeare-char
Overriding: device = cpu
number of parameters: 0.80M
Loading meta from data\shakespeare_char\meta.pkl...

Theren:
For But I fair pleadion, no do all thy Egal
Than heaves aginss were brey: not.

Sevire:
Nother is befy well he the honoun him the in the
Our the care as so for honou fall the wippea,
Which perte our thing him.

TLEONENIUS:
Say, go lies sir.

GLood, thou dow it are a their be.

CLOLIBET:
Hell not stort:
Theing wo not with in blay to seet a onde thess
And but hould the it rie storde carition word wound,
Pecarnury muspe more he hath unter the the to Give shy demay's
The say enter then the a
---------------

Rechild but to boftick true a bust sough;
Not be see in hand then hourse for deater,
And wee his, are have as a dest do of more,
That here you, this to my my spead: I parother
And that it the applain at and thy shall the be are
mistech, in ampin ouce the brow our my all before
A our to sorre then soul plesting speake lie
the have w

In [3]:
# evaluate metric: specific and general

# Specific metric: distribution of characters
def char_distribution(text):
    char_count = {}
    for char in text:
        if char in char_count:
            char_count[char] += 1
        else:
            char_count[char] = 1
    return char_count

# calculate the cosine similarity
def cosine_similarity(dict1, dict2):
    keys = set(dict1.keys()) | set(dict2.keys())
    dot_product = sum(dict1.get(k, 0) * dict2.get(k, 0) for k in keys)
    norm1 = sum(v ** 2 for v in dict1.values()) ** 0.5
    norm2 = sum(v ** 2 for v in dict2.values()) ** 0.5
    return dot_product / (norm1 * norm2)

In [4]:
# load training data
train_data_path = os.path.join("data", "shakespeare_char", "input.txt")
with open(train_data_path, "r") as f:
    train_data = f.read()
    
# load generated data
generated_data = result.stdout

# calculate character distribution
train_char_count = char_distribution(train_data)
generated_char_count = char_distribution(generated_data)

cosine_similarity(train_char_count, generated_char_count)

0.9860678780881484

In [29]:
# general metric: perplexity

In [1]:
# check loss in ckpt.pt
def check_loss(ckpt_path):
    checkpoint = torch.load(ckpt_path)
    return checkpoint['best_val_loss']

In [5]:
ckpt_path = os.path.join(out_dir, "ckpt.pt")
loss = check_loss(ckpt_path)
print(f"Best validation loss: {loss}")

Best validation loss: 1.8925344944000244


In [6]:
#params = " --compile=False --eval_iters=20 --log_interval=1 --block_size=64 --batch_size=12 --n_layer=4 --n_head=4 --n_embd=128 --max_iters=2000 --lr_decay_iters=2000 --dropout=0.0"
n_layers = [2, 4]
n_heads = [2, 4]

for n_layer in n_layers:
    for n_head in n_heads:
        if n_layer == 4 and n_head == 4:
            continue
        params = f" --n_layer={n_layer} --n_head={n_head}"
        print(f"Running with n_layer={n_layer}, n_head={n_head}")
        train_script = os.path.join("train.py")
        config_path = os.path.join("config", "train_shakespeare_char.py")
        command = f'python "{train_script}" "{config_path}" --device=cpu --compile=False --eval_iters=20 --log_interval=1 --block_size=64 --batch_size=12 --n_embd=128 --max_iters=2000 --lr_decay_iters=2000 --dropout=0.0' + params
        result = subprocess.run(command, shell=True, text=True, capture_output=True)
        # get loss
        ckpt_path = os.path.join(out_dir, "ckpt.pt")
        loss = check_loss(ckpt_path)
        print(f"Best validation loss: {loss}")

Running with n_layer=2, n_head=2
Best validation loss: 1.8904390335083008
Running with n_layer=2, n_head=4
Best validation loss: 1.9006503820419312
Running with n_layer=4, n_head=2
Best validation loss: 1.897631049156189


In [4]:
with open(os.path.join("data", "mydata_char", "笑林广记.txt"), "r") as f:
    data = f.read()
    
data = data[:80000]
len(data)

80000

In [5]:
# train another dataset
datalen = len(data)
i_list = []
cossims = []
losses = []

for i in range(0, datalen, 10000):
    mydata = data[:i+10000]
    with open(os.path.join("data", "mydata_char", "mydata.txt"), "w") as f:
        f.write(mydata)
    
    prepare_script = os.path.join("data", "mydata_char", "prepare.py")

    # Run the script using os.system
    os.system(f'python "{prepare_script}"')

    # Run scripts
    train_script = os.path.join("train.py")
    config_path = os.path.join("config", "train_mydata_char.py")
    command = f'python "{train_script}" "{config_path}" --device=cpu --compile=False --eval_iters=20 --log_interval=1 --block_size=64 --batch_size=12 --n_layer=4 --n_head=4 --n_embd=128 --max_iters=2000 --lr_decay_iters=2000 --dropout=0.0'
    result = subprocess.run(command, shell=True, text=True, capture_output=True)
    
    # Generate text by sample.py
    sample_script = os.path.join("sample.py")
    out_dir = "out-mydata-char"
    command = f'python "{sample_script}" --out_dir="{out_dir}" --device=cpu'
    result = subprocess.run(command, shell=True, text=True, capture_output=True)
    print(result.stdout.split("\n")[:5])
    
    # evaluate metric: specific
    # load training data
    train_data_path = os.path.join("data", "mydata_char", "mydata.txt")
    with open(train_data_path, "r") as f:
        train_data = f.read()
        
    # load generated data
    generated_data = result.stdout

    # calculate character distribution
    train_char_count = char_distribution(train_data)
    generated_char_count = char_distribution(generated_data)

    cossim = cosine_similarity(train_char_count, generated_char_count)
    
    print(f"i={i}, cosine similarity={cossim}")
    
    # loss
    ckpt_path = os.path.join(out_dir, "ckpt.pt")
    loss = check_loss(ckpt_path)
    
    i_list.append(i)
    cossims.append(cossim)
    losses.append(loss)

['Overriding: out_dir = out-mydata-char', 'Overriding: device = cpu', 'number of parameters: 0.96M', 'Loading meta from data\\mydata_char\\meta.pkl...', '']
i=0, cosine similarity=0.9162283876219303
['Overriding: out_dir = out-mydata-char', 'Overriding: device = cpu', 'number of parameters: 1.02M', 'Loading meta from data\\mydata_char\\meta.pkl...', '']
i=10000, cosine similarity=0.9086405265711462
['Overriding: out_dir = out-mydata-char', 'Overriding: device = cpu', 'number of parameters: 1.06M', 'Loading meta from data\\mydata_char\\meta.pkl...', '']
i=20000, cosine similarity=0.8981946113736679
['Overriding: out_dir = out-mydata-char', 'Overriding: device = cpu', 'number of parameters: 1.09M', 'Loading meta from data\\mydata_char\\meta.pkl...', '']
i=30000, cosine similarity=0.9157847241087812
['Overriding: out_dir = out-mydata-char', 'Overriding: device = cpu', 'number of parameters: 1.11M', 'Loading meta from data\\mydata_char\\meta.pkl...', '']
i=40000, cosine similarity=0.942305

In [9]:
# plot
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=i_list, y=cossims, mode="lines+markers", name="cosine similarity")
)
fig.update_layout(
    title="Cosine similarity over training data character size",
    xaxis_title="Characters",
    yaxis_title="Value",
    width=600,
    height=600,
)
fig.show()

In [10]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=i_list, y=losses, mode="lines+markers", name="loss")
)
fig.update_layout(
    title="Loss over training data character size",
    xaxis_title="Characters",
    yaxis_title="Value",
    width=600,
    height=600,
)
fig.show()